# Singleton

In [ ]:
class Singleton(object):

  # Método que é executado antes do init(construtor), ele cria um novo objeto
  def __new__(cls):

    # Se não existir o atributo instance
    if not hasattr(cls, 'instance'):

      # Cria o atributo instance e intancia o método pelo "new"
      cls.instance = super(Singleton, cls).__new__(cls)
      print(f'Creating the object {cls.instance}')
    return cls.instance

s1 = Singleton()
print(f'Instance 1: {id(s1)}')

s2 = Singleton()
print(f'Instance 2: {id(s2)}')

Creating the object <__main__.Singleton object at 0x7a53a4f66860>
Instance 1: 134499668486240
Instance 2: 134499668486240


# Usando Lazy Instance no Singleton

In [ ]:
# Fazendo uso da lazy instance
# Instancia um objeto que não é utilizado no momento da instanciação, ele é criado só quando precisar ser utilizado

class Singleton:
  __instance = None

  def __init__(self):

    # Se a instância do Singleton for None
    if not Singleton.__instance:
      print("The method __init"" was called...")

    # Se a instância não for None
    else:
      print(f"The instance is already created: {self.get_instance()}")

  # Método estático de classe
  @classmethod
  def get_instance(cls):

    # Se a instância for nula
    if not cls.__instance:

      # Instancia a classe
      cls.__instance = Singleton()
    return cls.__instance

s1 = Singleton() # Classe inicializada mas objeto não é criado

print(f"Object created now: {Singleton.get_instance()}")

s2 = Singleton() # Instância já criada

# Init é chamado duas vezes, justamente por não criar o objeto no momento da primeira instância

# Padrão lazy é utilizado com generators(não tem o objeto quando executa, só quando precisa)


The method __init was called...
The method __init was called...
Object created now: <__main__.Singleton object at 0x7a53a410dc00>
The instance is already created: <__main__.Singleton object at 0x7a53a410dc00>


# Singleton Monostate

In [ ]:
class Monostate:
  # Atributo de classe
  __state = {}

  # Construtor do objeto
  def __new__(cls, *args, **kwargs):
    # Construção do objeto
    obj = super(Monostate, cls).__new__(cls, *args, **kwargs)

    # Atribui um dicionário(objeto estruturado) ao estado
    obj.__dict__ = cls.__state
    return obj

m1 = Monostate()
print(f"M1 ID: {id(m1)}")
print(m1.__dict__)

m2 = Monostate()
print(f"M2 ID: {id(m2)}")
print(m2.__dict__)

m1.name = "Victor"
print(f"M1: {m1.__dict__}")
print(f"M2: {m2.__dict__}")

M1 ID: 136880959317680
{}
M2 ID: 136880959308992
{}
M1: {'name': 'Victor'}
M2: {'name': 'Victor'}


# Singleton e Metaclasses

In [ ]:
class University(type):

  # Método que é chamado quando o objeto precisa ser criado para uma classe existente
    # Quando instancia a Geek, o call da metaclasse é chamado
  def __call__(cls, *args, **kwargs):
    print(f"=== These are the args: {args}")
    return type.__call__(cls, *args, **kwargs)

class Geek(metaclass = University):
  def __init__(self, val1, val2):
    self.val1 = val1
    self.val2 = val2

object = Geek(42, 23)
print(object)

# Mais controle sobre criação da classe e instanciação de objetos
  # Logo, metaclasses podem ser utilizadas para criar Singletons

=== These are the args: (42, 23)


In [ ]:
class MetaSingleton(type):
  __instances = {}

  def __call__(cls, *args, **kwargs):

    # Se a classe não estiver dentro do conjunto de instâncias
    if cls not in cls.__instances:

      # Instancia um objeto e adiciona ao dicionário
      cls.__instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)

    # Retorna se ela já existe
    return cls.__instances[cls]

# Metaclasse passa a controlar a criação de Singletons(instanciação de objetos)
class Logger(metaclass=MetaSingleton):
  pass

log1 = Logger()
print(f"Log 1: {id(log1)}")

log2 = Logger()
print(f"Log 2: {id(log2)}")

Log 1: 136880048047872
Log 2: 136880048047872


# Projeto Real Singleton - 1

Aplicação Web que envolve várias operação de Read/Write em um banco de dados. Essa aplicação é dividida em vários serviços que executam as operações no banco de dados e os usuários interagem com uma interface que se comunica com uma API, que em algum momento executa as operaçõse no bando de dados(controlado por vários serviços).

Nesse sentido, o sistema precisa ter consistência entre as operações do banco de dados(evitar conflitos) e a utilização de memória e CPU devem estar otimizadas par ao tratamento de várias operações do banco de dados.

In [ ]:
import sqlite3

# Classe que herda de type, tem uma metaclasse
class Singleton(type):

  __instances = {}

  def __call__(cls, *args, **kwargs):
    if cls not in cls.__instances:
      cls.__instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
    return cls.__instances[cls]


# Utilização da classe Singleton para que tenha só uma instância de Database
class Database(metaclass=Singleton):

  # Atributo de classe inicializado como None
  connection = None

  # Método que verifica se a conexão é none
  def connect(self):

    # Se for None, ele cria um db na root e cria um cursor que permite acesso a operações no db
    if self.connection is None:
      print(f"We don't have a connection yet... let's create it")
      self.connection = sqlite3.connect('db.project')
      self.cursor = self.connection.cursor()
    return self.cursor

db1 = Database().connect()
db2 = Database().connect()

# db1.execute('SELECT * FROM produtos')

We don't have a connection yet... let's create it


# Projeto Real Singleton - 2

Atualmente existe a profissão de DevOps, imagine que sua função como DevOps é garantir a sanidade de servidores da empresa que trabalha. Para isso, quer que exista apenas uma instância de uma classe Health Check, que o objeto vai checar se todos os servidores estão operando.

A classe precisa ter uma lista de servidores para realizar a verificação, se um servidor foi removido da lista, o software deve detectar uma mudança e removê-lo dos servidores configurados para serem verificados.

In [ ]:
class HealthCheck:

  __instance = None

  def __new__(cls, *args, **kwargs):
    if not HealthCheck.__instance:
      HealthCheck.__instance = super(HealthCheck, cls).__new__(cls, *args, **kwargs)
    return HealthCheck.__instance

  def __init__(self):
    self.__servers = []

  def check_server(self, srv):
    print(f"Checking the {self.__servers[srv]}")
    # Aqui poderia fazer um ping para os servidores para verificar

  def add_server(self):
    self.__servers.append("NodeJS Server")
    self.__servers.append("Rust Server")
    self.__servers.append(".NET Server")
    self.__servers.append("Ruby Server")

  def change_server(self):
    self.__servers.pop()
    self.__servers.append("Flask Server")


hc1 = HealthCheck()
hc2 = HealthCheck()

hc1.add_server()
print("Health Check Timeline 1")
print("-"*30)
for srv in range(4):
  hc1.check_server(srv)

print("-"*30)
hc2.change_server()
print("Health Check Timeline 2")
print("-"*30)
for srv in range(4):
  hc2.check_server(srv)

Health Check Timeline 1
------------------------------
Checking the NodeJS Server
Checking the Rust Server
Checking the .NET Server
Checking the Ruby Server
------------------------------
Health Check Timeline 2
------------------------------
Checking the NodeJS Server
Checking the Rust Server
Checking the .NET Server
Checking the Flask Server


# Simple Factory

In [ ]:
from abc import ABCMeta, abstractmethod

# Extende uma metaclasse
class Animal(metaclass=ABCMeta):
  @abstractmethod
  def talk(self):
    pass

# Extende a classe Animal
class Dog(Animal):

  # Utiliza o método da classe animal
  def talk(self):
    print("Au au!")

class Cat(Animal):

  # Utiliza o método da classe animal
  def talk(self):
    print("Meow!")

# Fábrica
class SimpleFactory:

  # Método que cria um tipo de animal
  def create_talker_animal(self, type):

    # Instancia um objeto e executa um método
    return eval(type)().talk() # Eval serve para avaliar a string passada como parâmetro e tentar executar como comando python
    # Devolvendo só o Objeto
    #return eval(type)()

# Client
if __name__ == '__main__':
  factory = SimpleFactory()
  animal = input("Which animal you want to talk? [Dog, Cat]")
  factory.create_talker_animal(animal)

# Exemplo de Eval
  # eval(string)(instancia).método()
  # Pega a string e tenta executar como comando Python(ex: Cat())
  # O () efetua a instância daquele comando Python
  # O método é instanciado
  # eval("Cat")().talk()

# Tudo é feito a partir da fábrica, de forma que o cliente acessa somente a Fabric, que cria uma instância dos Animais
# Cliente não precisa saber o que acontece pois ele não acessa diretamente a classe a qual interagiu
# Ele interage via Factory
# A metaclasse abstrata é instanciada em outras classes
# A Factory acessa os métodos das classes

Which animal you want to talk? [Dog, Cat]Dog
Au au!


# Factory Method
Imagine que tem uma aplicação web que cria perfis em redes sociais automaticamente.

Exemplo: empresa que gerencia cadastros e perfis em redes sociais, alimentando as seções de acordo com as escolhas dos indivíduos.

In [ ]:
from abc import ABCMeta, abstractmethod

# Classe abstrata que vai servir de base para as outras classes
class Section(metaclass=ABCMeta):

  # Método de representação -> É abstrato pois as classes filhas que vão executar
  @abstractmethod
  def __repr__(self):
    pass

# Instancia o objeto
class PersonalSection(Section):
  def __repr__(self):
    return "Personal Section"

class AlbumSection(Section):
  def __repr__(self):
    return "Album Section"

class ProjectSection(Section):
  def __repr__(self):
    return "Project Section"

class PublicationSection(Section):
  def __repr__(self):
    return "Publication Section"

class Profile(metaclass=ABCMeta):

  # Cada perfil tem uma lista de seções e um método de criar perfil
  def __init__(self):
    self.sections = []
    self.create_profile()

  # Método abstrato pois as classes filhas que vão executar
  @abstractmethod
  def create_profile(self): # Esse aqui é o Factory Method
    pass

  def get_sections(self):
    return self.sections

  def add_section(self, section):
    self.sections.append(section)

class Linkedln(Profile):

  def create_profile(self):
    self.add_section(PersonalSection()) # Adiciona instância das seções na lista de seções
    self.add_section(ProjectSection())
    self.add_section(PublicationSection())

class Facebook(Profile):

  def create_profile(self):
    self.add_section(PersonalSection())  # Adiciona instância das seções na lista de seções
    self.add_section(AlbumSection())

if __name__ == "__main__":
  social_media = input("Which social media you want to create your profile? [Linkedln, Facebook] ")

  # A partir do profile, utiliza a instância e a executa em seguida
  profile = eval(social_media)()

  print(f"Creating profile on {type(profile).__name__}")
  print(f"The profile has the sections: {profile.get_sections()}")

# O cliente nunca interage diretamente com as seções, mas sim com um creator e concrete creator

# Classe está sendo criada de forma abstrata em Perfil(Creator)
# Métodos dela são utilizados pelas classes filhas Linkedin e Facebook(Concrete Creator)

Which social media you want to create your profile? [Linkedln, Facebook]Facebook
Creating profile on Facebook
The profile has the sections: [Personal Section, Album Section]


# Abstract Factory
**Contexto:** Imagine que está desenvolvendo um sistema para sua pizzaria favorita, que serve pizzas brasileiras, italianas, veganas e não veganas

In [ ]:
from abc import ABCMeta, abstractmethod

# Abstract Factory
class PizzaFactory(metaclass=ABCMeta):
  @abstractmethod
  def create_veggie_pizza(self):
    pass

  @abstractmethod
  def create_pizza(self):
    pass

# Concrete Factory A
class BrazilianPizza(PizzaFactory):
  def create_veggie_pizza(self):
    return LettucePizza()

  def create_pizza(self):
    return PepperoniPizza()

# Concrete Factory B
class ItalianPizza(PizzaFactory):
  def create_veggie_pizza(self):
    return BroccoliPizza()

  def create_pizza(self):
    return BolognaPizza()

# Abstract Product A
class VeggiePizza(metaclass=ABCMeta):
  @abstractmethod
  def prepare(self, VeggiePizza):
    pass

# Abstract Product B
class Pizza(metaclass=ABCMeta):
  @abstractmethod
  def serve(self, VeggiePizza):
    pass

# Concrete Products
class LettucePizza(VeggiePizza):
  def prepare(self):
    print(f"Preparing {type(self).__name__}")


class PepperoniPizza(Pizza):
  def serve(self, VeggiePizza):
    print(f"{type(self).__name__} is served with Pepperoni in {type(VeggiePizza).__name__}")

class BroccoliPizza(VeggiePizza):
  def prepare(self):
    print(f"Preparing {type(self).__name__}")

class BolognaPizza(Pizza):
  def serve(self, VeggiePizza):
    print(f"{type(self).__name__} is served with Bologna in {type(VeggiePizza).__name__}")

# Client
class Pizzeria:

  def make_pizzas(self):
    for factory in [BrazilianPizza(), ItalianPizza()]:
      self.factory = factory
      self.pizza = self.factory.create_pizza()
      self.veggie_pizza = self.factory.create_veggie_pizza()
      self.veggie_pizza.prepare()
      self.pizza.serve(self.veggie_pizza)

pizzeria = Pizzeria()
pizzeria.make_pizzas()

Preparing LettucePizza
PepperoniPizza is served with Pepperoni in LettucePizza
Preparing BroccoliPizza
BolognaPizza is served with Bologna in BroccoliPizza
